In [10]:
# import the required libraries
import os
import re
import csv
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_pred = folderlink+input_folder+"new_predikanten.csv"

In [3]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
jaar_vertrek_type = {'jaar vertrek': pd.Int64Dtype(), 'ind_id': pd.Int64Dtype(),'dag intrede': pd.Int64Dtype(), 'dag vertrek': pd.Int64Dtype() }
df = pd.read_csv(input_pred, sep=',', dtype= jaar_vertrek_type, encoding='utf-8')

In [5]:
df.head(10)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
0,35651,1,ZH,NaN,Oudewater,NaN,(Hekendorp) Brunner; L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,NaN
2,7179,3,NB,NaN,Breda,NaN,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
3,21505,3,GR,NaN,Hornhuizen,NaN,Aa; dr. P.J.B.K. Simon van der,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN
4,36887,3,NH,NaN,Purmerend,NaN,Aa; dr. P.J.B.K. Simon van der,Goutum,<NA>,NaN,1886,Winterswijk,<NA>,NaN,1892,NaN
5,50146,3,GE,NaN,Winterswijk,NaN,Aa; dr. P.J.B.K. Simon van der,Purmerend,<NA>,NaN,1892,Breda,<NA>,NaN,1893,NaN
6,15179,3,FR,NaN,Goutum,NaN,Aa; dr. P.J.B.K. van der,Kloosterburen,<NA>,NaN,1866,Purmerend,<NA>,NaN,1886,NaN
7,7204,3,NB,NaN,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
8,19779,4,GE,NaN,Herwijnen,NaN,Aa; W.P. van der,kandidaat,24,december,1989,NaN,<NA>,NaN,<NA>,NaN
9,35941,5,OV,NaN,Overdinkel,NaN,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,NaN


In [6]:
# The dataset is split in two. Since the dataset is composed of rows that have information about where someone went to ('vertrek naar of vanwege') and where he or she came from ('Herkomst'), which is foremost for ministers from after 1816, and rows that have only a year when they move position a cut was made based on the field Herkomst having values NaN.
# The former is named dm_part2, the latter dm_part1. 

dm_part1 = df[df['Herkomst'].isna()]

In [7]:
dm_part1.head(10)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
36,35685,<NA>,NH,Alkmaar,Oudkarspel,NaN,Aalburg; Johannes van,NaN,<NA>,NaN,1743,NaN,<NA>,NaN,1777,NaN
62,22489,<NA>,NH,Woerden,Kalslagen,NaN,Aalst; Cornelius van,NaN,<NA>,NaN,1717,NaN,<NA>,NaN,1751,NaN
68,46953,<NA>,GE,Zaltbommel,Vuren,NaN,Aalst; Gerardus van,NaN,<NA>,NaN,1704,NaN,<NA>,NaN,1706,NaN
69,41854,<NA>,ZH,Schouwen-Duiveland,Sommelsdijk,NaN,Aalst; Gerardus van,NaN,<NA>,NaN,1706,NaN,<NA>,NaN,1715,NaN
70,48641,<NA>,NH,Haarlem,West-Zaandam,NaN,Aalst; Gerardus van,NaN,<NA>,NaN,1715,NaN,<NA>,NaN,1755,NaN
71,741,<NA>,ZE,Walcheren,Aardenburg,NaN,Aalst; Wilhelmus,NaN,<NA>,NaN,1695,NaN,<NA>,NaN,1700,NaN
72,8276,<NA>,NH,Haarlem,Castricum,NaN,Aalstius; Henricus,NaN,<NA>,NaN,1700,NaN,<NA>,NaN,1733,NaN
73,32102,<NA>,NH,Hoorn,Obdam,NaN,Aalstius; Johannes,NaN,<NA>,NaN,1644,NaN,<NA>,NaN,1648,NaN
74,6933,<NA>,NB,’s-Hertogenbosch,Boxtel,NaN,Aalstius; Johannes,NaN,<NA>,NaN,1648,NaN,<NA>,NaN,1652,NaN
75,32398,<NA>,NB,Peel- en Kempenland,Oirschot,NaN,Aalstius; Johannes,NaN,<NA>,NaN,1648,NaN,<NA>,NaN,1657,NaN


In [8]:
dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
dm_part1['pred_year_end'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar vertrek'].astype(str)

C:\Windows\Temp\ipykernel_1348\1813838777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_1348\1813838777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
C:\Windows\Temp\ipykernel_1348\1813838777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [9]:
tdf = dm_part1
variable_end = 'pred_year_end'
variable_start = 'pred_year_start'
variable_name = 'predikant'

In [37]:
def create_node_paths_dataframe(dataframe):
    # Generate the directed graph G
    G = nx.DiGraph()
    for _, row in dataframe.iterrows():
        G.add_edge(row[variable_start], row[variable_end], name=row[variable_name])

    # Find connected components and assign path IDs
    paths = list(nx.connected_components(G.to_undirected()))
    path_ids = list(range(1, len(paths) + 1))
    node_path_pairs = []
    for path_id, path_nodes in zip(path_ids, paths):
        node_path_pairs.extend([(node, path_id) for node in path_nodes])

    # Create a DataFrame from the node-path pairs
    node_paths = pd.DataFrame(node_path_pairs, columns=[variable_start, 'individual'])

    # Join the original DataFrame with the node-paths DataFrame
    joined = pd.merge(dataframe, node_paths, left_on=variable_start, right_on=variable_start, how='left')

    return joined

In [38]:
result = create_node_paths_dataframe(tdf)

In [39]:
unique_individuals_table_p1 = result[[variable_name,'individual']].copy()

In [40]:
unique_individuals_table_p1 = unique_individuals_table_p1.drop_duplicates(subset=['individual'])

In [41]:
unique_individuals_table_p1.describe()

,individual
count,13956.000000
mean,6978.500000
std,4028.894513
min,1.000000
25%,3489.750000
50%,6978.500000
75%,10467.250000
max,13956.000000


In [42]:
unique_individuals_table_p1.head(150)

,predikant,individual
0,Aalburg;Johannesvan,1
1,Aalst;Corneliusvan,2
2,Aalst;Gerardusvan,3
5,Aalst;Wilhelmus,4
6,Aalstius;Henricus,5
7,Aalstius;Johannes,6
11,Aalstius;Johannes,7
14,Aalstius;Leonardus,8
16,Aalstius;Petrus,9
19,Aalstius;Wilhelmus,10


In [31]:
dm_part2 = df[df['Herkomst'].notna()]

In [32]:
dm_part2.head(50)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
0,35651,1,ZH,NaN,Oudewater,NaN,(Hekendorp) Brunner; L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,NaN
2,7179,3,NB,NaN,Breda,NaN,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
3,21505,3,GR,NaN,Hornhuizen,NaN,Aa; dr. P.J.B.K. Simon van der,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN
4,36887,3,NH,NaN,Purmerend,NaN,Aa; dr. P.J.B.K. Simon van der,Goutum,<NA>,NaN,1886,Winterswijk,<NA>,NaN,1892,NaN
5,50146,3,GE,NaN,Winterswijk,NaN,Aa; dr. P.J.B.K. Simon van der,Purmerend,<NA>,NaN,1892,Breda,<NA>,NaN,1893,NaN
6,15179,3,FR,NaN,Goutum,NaN,Aa; dr. P.J.B.K. van der,Kloosterburen,<NA>,NaN,1866,Purmerend,<NA>,NaN,1886,NaN
7,7204,3,NB,NaN,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
8,19779,4,GE,NaN,Herwijnen,NaN,Aa; W.P. van der,kandidaat,24,december,1989,NaN,<NA>,NaN,<NA>,NaN
9,35941,5,OV,NaN,Overdinkel,NaN,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,NaN


In [33]:
dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')

dm_part2['gemeente'] = dm_part2['gemeente'].str[:6]
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str[:6]


C:\Windows\Temp\ipykernel_1348\541579337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_1348\541579337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_1348\541579337.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [34]:
dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
dm_part2['pred_end'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['vertrek naar of vanwege'].astype(str)+'_'+dm_part2['jaar vertrek'].astype(str)

C:\Windows\Temp\ipykernel_1348\4125051709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_1348\4125051709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
C:\Windows\Temp\ipykernel_1348\4125051709.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [35]:
dm_part2 = dm_part2.sort_values(by='pred_start')

In [43]:
tdf = dm_part2
variable_end = 'pred_end'
variable_start = 'pred_start'
variable_name = 'predikant'

In [44]:
result_dmpart2 = create_node_paths_dataframe(tdf)

In [45]:
unique_individuals_table_p2 = result_dmpart2[[variable_name,'individual']].copy()

In [46]:
unique_individuals_table_p2 = unique_individuals_table_p2.drop_duplicates(subset=['individual'])

In [47]:
unique_individuals_table_p2.describe()

,individual
count,17700.000000
mean,8850.500000
std,5109.694218
min,1.000000
25%,4425.750000
50%,8850.500000
75%,13275.250000
max,17700.000000


In [48]:
unique_individuals_table_p2.head(50)

,predikant,individual
0,'tHofop;W.J.,1
1,(Hekendorp)Brunner;L.,2
2,(Hiltenvan-)Matthijsen;mevrouwL.,3
3,Aa;W.P.vander,4
4,Aa;dr.P.J.B.K.S.vander,5
5,Aa;dr.P.J.B.K.Simonvander,6
6,Aa;dr.P.J.B.K.Simonvander,7
8,Aa;dr.P.J.B.K.vander,8
9,Aa;drPieterJanBaptistKarelSimonvander,9
10,Aaij;J.,10


In [49]:
result_dmpart2.head(50)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,pred_start,pred_end,individual
0,18099,<NA>,GE,NaN,Hedel,NaN,'tHofop;W.J.,kandidaat,1,januari,1973,Ouddor,20,december,1978,NaN,'tHofop;W.J._Hedel_1973,'tHofop;W.J._Ouddor_1978,1
1,35651,1,ZH,NaN,Oudewa,NaN,(Hekendorp)Brunner;L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN,(Hekendorp)Brunner;L._Oudewa_1867,(Hekendorp)Brunner;L._Lobith_1868,2
2,45546,2,GE,NaN,Velp,2. Oude Jan,(Hiltenvan-)Matthijsen;mevrouwL.,kandidaat,2,maart,1986,Apeldo,23,oktober,1994,NaN,(Hiltenvan-)Matthijsen;mevrouwL._Velp_1986,(Hiltenvan-)Matthijsen;mevrouwL._Apeldo_1994,3
3,19779,4,GE,NaN,Herwij,NaN,Aa;W.P.vander,kandidaat,24,december,1989,NaN,<NA>,NaN,<NA>,NaN,Aa;W.P.vander_Herwij_1989,Aa;W.P.vander_nan_<NA>,4
4,7204,3,NB,NaN,Breda,4. Wijk Noord,Aa;dr.P.J.B.K.S.vander,Winterswijk,26,februari,1893,emerit,1,mei,1904,NaN,Aa;dr.P.J.B.K.S.vander_Breda_1893,Aa;dr.P.J.B.K.S.vander_emerit_1904,5
5,21505,3,GR,NaN,Hornhu,NaN,Aa;dr.P.J.B.K.Simonvander,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN,Aa;dr.P.J.B.K.Simonvander_Hornhu_1862,Aa;dr.P.J.B.K.Simonvander_Goutum_1866,6
6,36887,3,NH,NaN,Purmer,NaN,Aa;dr.P.J.B.K.Simonvander,Goutum,<NA>,NaN,1886,Winter,<NA>,NaN,1892,NaN,Aa;dr.P.J.B.K.Simonvander_Purmer_1886,Aa;dr.P.J.B.K.Simonvander_Winter_1892,7
7,50146,3,GE,NaN,Winter,NaN,Aa;dr.P.J.B.K.Simonvander,Purmerend,<NA>,NaN,1892,Breda,<NA>,NaN,1893,NaN,Aa;dr.P.J.B.K.Simonvander_Winter_1892,Aa;dr.P.J.B.K.Simonvander_Breda_1893,7
8,15179,3,FR,NaN,Goutum,NaN,Aa;dr.P.J.B.K.vander,Kloosterburen,<NA>,NaN,1866,Purmer,<NA>,NaN,1886,NaN,Aa;dr.P.J.B.K.vander_Goutum_1866,Aa;dr.P.J.B.K.vander_Purmer_1886,8
9,7179,3,NB,NaN,Breda,NaN,Aa;drPieterJanBaptistKarelSimonvander,Winterswijk,26,februari,1893,emerit,1,mei,1904,NaN,Aa;drPieterJanBaptistKarelSimonvander_Breda_1893,Aa;drPieterJanBaptistKarelSimonvander_emerit_1904,9
